Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name, r-number, collaborators, and the generative AI statement below:

In [ ]:
# fill in your name, r-number, and collaborators (if any)
NAME = ""
r_number=""
COLLABORATORS = ""

# gen-AI (LLM) statement. In case large language models (generative AI systems such as chatGPT, co-pilot, ...) were used to complete this notebook, 
# fill in which models were used and what for
gen_AI_statement = ""

---

In [ ]:
# import modules needed for this notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import matplotlib.cm as cm

print(f"Numpy version      : {np.__version__}")
print(f"Matplotlib version : {matplotlib.__version__}")

---
## Motivation & Aim

In nuclear medicine physics, many reconstruction and quantification problems ultimately reduce to **estimating unknown parameters from noisy counting data**. Because the measurements are counts, the natural statistical model is the **Poisson distribution**, and one of the most widely used estimation strategies is the **maximum likelihood estimator (MLE)**.  

The concept of MLE is e.g. used when we reconstruct PET images using the iteratice MLEM algorithm. 
MLEM converges to the MLE (maximum likelihood image) given a set of count measurements along different line of response as shown below.



In [ ]:
figp, axp = plt.subplots(figsize=(6, 6))
axp.set_aspect('equal')
axp.axis('off')

# --- Parameters ---
n = 4
cell = 1.0
half = n * cell / 2
R_min = (2**0.5) * half
R = R_min + 0.35
N_det = 36
theta0 = 0.0

# --- Pixel grid ---
x0, y0 = -half, -half
k = 1
for j in range(n):
    for i in range(n):
        axp.add_patch(Rectangle((x0 + i*cell, y0 + j*cell), cell, cell,
                                fill=False, linewidth=1.5, edgecolor='black'))
        cx = x0 + (i + 0.5) * cell
        cy = y0 + (j + 0.5) * cell
        axp.text(cx, cy, rf'$x_{{{k}}}$', ha='center', va='center', fontsize=11)
        k += 1

# Outer border
axp.add_patch(Rectangle((x0, y0), n*cell, n*cell,
                        fill=False, linewidth=1.8, edgecolor='black'))

# --- Detector ring ---
axp.add_patch(Circle((0, 0), R, fill=False, linewidth=3, edgecolor='black'))
angles = theta0 + 2*np.pi*np.arange(N_det)/N_det
xs = R * np.cos(angles)
ys = R * np.sin(angles)
axp.scatter(xs, ys, s=28, color='#f39c12', zorder=3)

# Label every 4th detector
for m in range(0, N_det, 4):
    axp.text((R + 0.18) * np.cos(angles[m]),
             (R + 0.18) * np.sin(angles[m]),
             f'$d_{{{m}}}$', ha='center', va='center', fontsize=10)

# --- Fixed LOR pairs ---
pairs = [(10, 22), (2, 18), (13, 27), (6, 30)]
labels = [r'$y_1$', r'$y_2$', r'$y_3$', r'$y_4$']
lor_color = cm.tab10(1)

for (p, q), lab in zip(pairs, labels):
    x1, y1 = xs[p], ys[p]
    x2, y2 = xs[q], ys[q]
    axp.plot([x1, x2], [y1, y2], linewidth=1.8, color=lor_color, zorder=2)


axp.set_xlim(-R - 0.6, R + 0.6)
axp.set_ylim(-R - 0.6, R + 0.6)


An important property of any estimator is whether it is **biased**: does its expected value equal the true underlying parameter, or does it systematically over- or under-estimate? Understanding bias (related accuracy) is crucial, systematic bias can lead to incorrect conclusions.
A 2nd important propert is **variance** (related called precision). 
Ideally, we want an unbias estimator that has low variance.

In real PET reconstructions, where we reconstruct millions of voxels from count values along billions of LORs, finding the MLE requires iterative algorithms (e.g. MLEM).

That is why, in this notebook, we explore a **simplified toy forward model** consisting of 2 voxels and 3 LORs as shown below that captures the essence of more complex problems in emission tomography:

$$
\bar y_1 (x_1, x_2) = \alpha x_1,\qquad 
\bar y_2 (x_1, x_2) = \alpha x_2,\qquad 
\bar y_3 (x_1, x_2) = \alpha (x_1+x_2),
$$

where $\bar y_i$ are the expectations of **Poisson-distributed**  observed counts, $\alpha$ is a known system constant modeling acquisition time and detector sensitivity, and $x_1, x_2$ are the unknown parameters (pixel values) to be estimated.

This simplified toy model has the advantage that we can derive an analytical expression for the MLE avoiding lengthy iterative algorithms and is shown below.



In [ ]:
fige, axe = plt.subplots(figsize=(7, 4), layout='constrained')

voxel_size = 1.0
axe.add_patch(Rectangle((0, 0), voxel_size, voxel_size, fill=False, linewidth=2))
axe.text(0.2, 0.1, r"$x_1$", ha="center", va="center", fontsize="large")
axe.add_patch(Rectangle((voxel_size, 0), voxel_size, voxel_size, fill=False, linewidth=2))
axe.text(1.2, 0.1, r"$x_2$", ha="center", va="center", fontsize="large")

# LOR 1: vertical through voxel 1
axe.plot([0.5, 0.5], [-0.3, 1.3], linewidth=1.5)
axe.text(0.05, 1.35, r"$y_1 \sim \mathrm{Poisson}(\alpha x_1)$", va="bottom", color = plt.cm.tab10(0))

# LOR 2: vertical through voxel 2
axe.plot([1.5, 1.5], [-0.3, 1.3], linewidth=1.5)
axe.text(1.55, 1.35, r"$y_2 \sim \mathrm{Poisson}(\alpha x_2)$", va="bottom", color = plt.cm.tab10(1))

# LOR 3: horizontal through center of voxels (y=0.5)
axe.plot([-0.6, 2.6], [0.5, 0.5], linewidth=1.5, color="green")
axe.text(2.65, 0.55, r"$y_3 \sim \mathrm{Poisson}(\alpha(x_1+x_2))$", va="bottom", ha="left", color = plt.cm.tab10(2))

axe.set_aspect('equal')
axe.axis('off')


This assignment has two main goals:

1. **Analytical understanding**: Derive the closed-form MLE for $(x_1, x_2)$. This will give you practice in setting up and solving likelihood equations — an important skill that is useful when estimating parameters from measured data applicable in many different scientific fields.
2. **Computational experiment**: Use **Monte Carlo simulations** to test whether a given (non-trivial) estimator is biased. You will simulate many Poisson datasets (noise realizations of measurements), compute the estimates, and study whether their average matches the true parameters. Moreover, you will also estimate the variance of the estimator.

---

## Least squares estimator (LSE)

One possible (but maybe not optimal) estimator is the **least squares estimator (LSE)**, which minimizes the squared error ($SE$) between the observed ($y_i$) and expected counts ($\bar y_i$) given by
$$
SE(x_1, x_2) = \sum_{i=1}^3 (\bar y_i(x_1, x_2) - y_i)^2 \quad .
$$

$$
(\hat x_1^{LSE}, \hat x_2^{LSE}) = \arg\min_{x_1, x_2} SE(x_1, x_2).
$$

For this toy model, the LSE has the simple closed-form expressions
$$
\hat x_1^{LSE} = \frac{2y_1 + (y_3 - y_2)}{3\alpha},\qquad
\hat x_2^{LSE} = \frac{2y_2 + (y_3 - y_1)}{3\alpha} \ ,
$$

which we implement in two functions in the next code cell.
We will compare the properties of LSE against the MLE you derive in the next task.

In [ ]:
def lse_x1(alpha: float, y1: int, y2: int, y3: int) -> float:
    """Compute the least squares estimator for x1, 
    given the forward model above, the measurements y1, y2, y3 and system sensitivity alpha.
    """
    return (2*y1 + (y3 - y2))/(3*alpha)

def lse_x2(alpha: float, y1: int, y2: int, y3: int) -> float:
    """Compute the least squares estimator for x2, 
    """
    return (2*y2 + (y3 - y1))/(3*alpha)

---

## Part 1 - Maximum Likelihood Estimator (MLE) of the Poisson log-likelihood

Derive a closed-form expression for the maximum likelihood estimator (MLE) for $(x_1, x_2)$ as a function of $\alpha$, $y_1$, $y_2$, and $y_3$. 
The MLE is defined as the $(x_1, x_2)$ values that maximize the Poisson log-likelihood function:
$$
\log L(x_1, x_2) = \sum_{i=1}^3 \left( y_i \ln(\bar y_i(x_1, x_2)) - \bar y_i(x_1, x_2) - \ln y_i! \right).
$$

$$
(\hat x_1^{MLE}, \hat x_2^{MLE}) = \arg\max_{x_1, x_2} \log L(x_1, x_2).
$$

Implement both expressions in the python functions below. **Make sure that both functions also return meaningful values for the special case** $y_1 = 0$ and $y_2 = 0$. *Note: in the latter special case the MLE is not unique, choose it such that in this case, the MLE for $x_1$ and $x_2$ are the same.*

In [ ]:
def mle_x1(alpha: float, y1: int, y2: int, y3:int) -> float:
    """Compute the maximum likelihood estimator for x1, 
    given the forward model above assuming Poisson distributed measurements y1, y2, y3 and system sensitivity alpha.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# print the output of the MLE for x1 for different inputs
for a, b, c, d in ((0.5, 1, 3, 2), (2.5, 5, 7, 9), (1.5, 6, 2, 7), (0.5, 0, 0, 2)):
    print(f"alpha = {a}, y1 = {b}, y2 = {c}, y3 = {d}: MLE for x1 = {mle_x1(alpha = a, y1 = b, y2 = c, y3 = d)}")
    


In [ ]:
def mle_x2(alpha: float, y1: int, y2: int, y3:int) -> float:
    """Compute the maximum likelihood estimator for x2, 
    given the forward model above assuming Poisson distributed measurements y1, y2, y3 and system sensitivity alpha.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# print the output of the MLE for x2 for different inputs
for a, b, c, d in ((0.5, 1, 3, 2), (2.5, 5, 7, 9), (1.5, 6, 2, 7), (0.5, 0, 0, 2)):
    print(f"alpha = {a}, y1 = {b}, y2 = {c}, y3 = {d}: MLE for x2 = {mle_x2(alpha = a, y1 = b, y2 = c, y3 = d)}")


---

## Part 2 - Monte Carlso-based bias and variance estimation

Analytically calculating the bias and variance of the MLE is quite involved. (Why?) 

Numerically, we can estimate the bias and variance of any estimator using Monte Carlo simulations. The idea is as follows:
1. Choose a set of true parameters $(x_1^{\text{true}}, x_2^{\text{true}})$ and a system constant $\alpha$.
2. Compute the corresponding noise-free mean measurements $\bar y_i = \bar y_i(x_1^{\text{true}}, x_2^{\text{true}})$.
3. Simulate $N$ independent noise realizations of the measurements $y_i^{(j)} \sim \text{Poisson}(\bar y_i)$ for $j = 1, \ldots, N$.
4. For each noise realization, compute the estimates $(\hat x_1^{(j)}, \hat x_2^{(j)})$ using a given estimator (e.g., the MLE).
5. Estimate the bias and variance of the estimator using the sample mean and sample variance of the estimates:
   - Bias: $\text{Bias}(\hat x_i) \approx \frac{1}{N} \sum_{j=1}^N \hat x_i^{(j)} - x_i^{\text{true}}$ - hint: use `np.mean()` for the first term
   - Variance: $\text{Var}(\hat x_i) \approx \frac{1}{N-1} \sum_{j=1}^N (\hat x_i^{(j)} - \bar{\hat x_i})^2$, where $\bar{\hat x_i} = \frac{1}{N} \sum_{j=1}^N \hat x_i^{(j)}$ - hint: use `np.var(ddof = 1)`

*Note: For demonstration purposes (and short runtimes), we choose `N=int(1e6)` noise realizations below. Increasesing this value will yield more correct results.*


In [ ]:
np.random.seed(1)

# step 1: choose "true" values for alpha, x1, x2
alpha_test: float = 0.5
x1_true: float = 1.8
x2_true: float = 1.0

# step 2: compute the corresponding noise-free measurements
ybar_1 = alpha_test*x1_true
ybar_2 = alpha_test*x2_true
ybar_3 = alpha_test*(x1_true + x2_true)


# step 3: simulate N noisy measurements from the Poisson distribution
N = int(1e6)

y1_test = np.random.poisson(ybar_1, N)
y2_test = np.random.poisson(ybar_2, N)
y3_test = np.random.poisson(ybar_3, N)


First, we apply the Monte Carlo procedure to the LSE to numerically estimate the bias and variance of the LSE for the given true parameters and system constant. 

In [ ]:
# step 4: compute the estimators for each noisy measurement
#         here we use the above defined (naive) least squares estimator (LSE)

x1_lse = np.zeros(N)
x2_lse = np.zeros(N)

for i in range(N):
    x1_lse[i] = lse_x1(alpha_test, y1_test[i], y2_test[i], y3_test[i])
    x2_lse[i] = lse_x2(alpha_test, y1_test[i], y2_test[i], y3_test[i])

print(f"numeric expectation of LSEs - x1: {x1_lse.mean(): .5f}, x2: {x2_lse.mean(): .5f}")
print(f"numeric bias of LSEs        - x1: {(x1_lse.mean() - x1_true): .5f}, x2: {(x2_lse.mean() - x2_true): .5f}")
print(f"numeric variances of LSEs   - x1: {x1_lse.var(ddof=1): .5f}, x2: {x2_lse.var(ddof=1): .5f}")

Before doing the same for the above implemented MLE, we will visualize the distribution of LSEs using histograms and box plots.


In [ ]:
bowplot_kws = dict(
    positions=[1],
    widths=0.3,
    vert=False,
    showfliers=False,
    showmeans=True,
    meanline=True,
    medianprops=dict(linewidth=0),
    meanprops=dict(color='black', linestyle='-', linewidth=1.5))

fig1, ax1 = plt.subplots(
    2, 1, figsize=(8, 3), layout='constrained', sharex=True, sharey=True
)


# Bottom axis: histograms
bins = np.linspace(min(x1_lse.min(), x2_lse.min()), 1.01*max(x1_lse.max(), x2_lse.max()), 200)

ax1[0].hist(x1_lse, bins=bins, label='x1_lse', density=True, color ='C0')
ax1[0].axvline(x1_true, color='red', ls=':', lw = 1.0, label='true x1')
ax1[0].boxplot(x1_lse, **bowplot_kws)
ax1[1].hist(x2_lse, bins=bins, label='x2_lse', density=True, color ='C1')
ax1[1].axvline(x2_true, color='red', ls=':', lw = 1.0, label='true x2')
ax1[1].boxplot(x2_lse, **bowplot_kws)
ax1[0].legend()
ax1[1].legend()


ax1[1].set_xlabel('Estimator Value')
ax1[0].set_title('Histograms and boxplots of x1_lse and x2_lse', fontsize = "medium")
ax1[0].grid(ls=":")
ax1[1].grid(ls=":")


Now we repeat the Monte Carlo procedure for the MLE and compare the bias and variance of the MLE against the LSE.

In [ ]:
# step 4: compute the estimators for each noisy measurement
#         here we use the above defined maximum likelihood estimator (MLE)

x1_mle = np.zeros(N)
x2_mle = np.zeros(N)

for i in range(N):
    x1_mle[i] = mle_x1(alpha_test, y1_test[i], y2_test[i], y3_test[i])
    x2_mle[i] = mle_x2(alpha_test, y1_test[i], y2_test[i], y3_test[i])

print(f"numeric expectations of MLEs - x1: {x1_mle.mean(): .5f}, x2: {x2_mle.mean(): .5f}")
print(f"numeric bias of MSEs         - x1: {(x1_mle.mean() - x1_true): .5f}, x2: {(x2_mle.mean() - x2_true): .5f}")
print(f"numeric variances of MLEs    - x1: {x1_mle.var(ddof=1): .5f}, x2: {x2_mle.var(ddof=1): .5f}")

We also visualize the distribution of MLEs using histograms and box plots and compare them against the LSE.

In [ ]:
fig2, ax2 = plt.subplots(4, 1, figsize=(8, 6), layout='constrained', sharex=True, sharey=False)

# x1_lse
ax2[0].hist(x1_lse, bins=bins, label='x1_lse', density=True, color='C0')
ax2[0].axvline(x1_true, color='red', ls=':', lw=1.0, label='true x1')
ax2[0].boxplot(x1_lse, **bowplot_kws)
ax2[0].legend()
ax2[0].set_title('Histograms and boxplots of x1_lse, x1_mle, x2_lse, and x2_mle', fontsize = "medium")

# x1_mle
ax2[1].hist(x1_mle, bins=bins, label='x1_mle', density=True, color='C2')
ax2[1].axvline(x1_true, color='red', ls=':', lw=1.0, label='true x1')
ax2[1].boxplot(x1_mle, **bowplot_kws)
ax2[1].legend()

# x2_lse
ax2[2].hist(x2_lse, bins=bins, label='x2_lse', density=True, color='C1')
ax2[2].axvline(x2_true, color='red', ls=':', lw=1.0, label='true x2')
ax2[2].boxplot(x2_lse, **bowplot_kws)
ax2[2].legend()

# x2_mle
ax2[3].hist(x2_mle, bins=bins, label='x2_mle', density=True, color='C3')
ax2[3].axvline(x2_true, color='red', ls=':', lw=1.0, label='true x2')
ax2[3].boxplot(x2_mle, **bowplot_kws)
ax2[3].legend()

ax2[3].set_xlabel('Estimator Value')
for ax in ax2:
    ax.grid(ls=":")


## Part 3 - Analyzing and interpreting the simulation results

Answer the following True or False questions based on your interpretation of the results from this notebook by completing the following functions.

*Note: please only implement `return True` or `return False` to indicate whether the statement in the function names for our toy model using the values $\alpha = 0.5$, $x_1 = 1.8$, $x_2 = 1.0$ is true or false.*

In [ ]:
def x1_lse_has_more_than_2_percent_bias() -> bool:
    """Returns True if the relative bias of LSE for x1 is more than 2%, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x1_lse_has_more_than_2_percent_bias(): {x1_lse_has_more_than_2_percent_bias()}")


In [ ]:
def x2_lse_has_more_than_2_percent_bias() -> bool:
    """Returns True if the relative bias of LSE for x2 is more than 2%, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x2_lse_has_more_than_2_percent_bias(): {x2_lse_has_more_than_2_percent_bias()}")


In [ ]:
def x1_mle_has_more_than_2_percent_bias() -> bool:
    """Returns True if the relative bias of MLE for x1 is more than 2%, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x1_mle_has_more_than_2_percent_bias(): {x1_mle_has_more_than_2_percent_bias()}")


In [ ]:
def x2_mle_has_more_than_2_percent_bias() -> bool:
    """Returns True if the relative bias of MLE for x2 is more than 2%, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x2_mle_has_more_than_2_percent_bias(): {x2_mle_has_more_than_2_percent_bias()}")


In [ ]:
def x1_lse_has_smaller_variance_compared_to_x1_mle() -> bool:
    """Returns True if the variance of the LSE of x1 is smaller than the variance of the MLE of x1, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x1_lse_has_smaller_variance_compared_to_x1_mle(): {x1_lse_has_smaller_variance_compared_to_x1_mle()}")


In [ ]:
def x2_lse_has_smaller_variance_compared_to_x2_mle() -> bool:
    """Returns True if the variance of the LSE of x2 is smaller thant the variance of the MLE of x2, False otherwise."""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
print(f"x2_lse_has_smaller_variance_compared_to_x2_mle(): {x2_lse_has_smaller_variance_compared_to_x2_mle()}")
